In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from geopy.distance import geodesic
import sqlite3


In [2]:
SAMPLE_SIZE = 5000

# Load dataset
data = pd.read_csv('DataS1/jaguar_movement_data.csv')
conn = sqlite3.connect('jaguar_data.db')

data = data.sample(SAMPLE_SIZE) 

data.head()

C:\Users\lmars\AppData\Local\Temp\ipykernel_11772\3241155957.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('DataS1/jaguar_movement_data.csv')


,Event_ID,timestamp,location.long,location.lat,individual.taxon.canonical.name,tag.local.identifier,individual.local.identifier (ID),study.name,country
23020,23021.0,11/29/11 22:02,-56.329004,-19.959032,Panthera onca,31936,19,jaguar_Oncafari Project,Brazil
89025,89026.0,12/3/13 5:02,-57.491217,-16.860395,Panthera onca,34770,81,Jaguar_Taiama,Brazil
98722,98723.0,5/2/15 5:01,-57.384168,-16.986887,Panthera onca,34770,81,Jaguar_Taiama,Brazil
72098,72099.0,11/21/13 3:01,-56.263650,-19.881856,Panthera onca,33301,69,jaguar_Oncafari Project,Brazil
82859,82860.0,4/14/07 4:03,-60.417902,-20.482090,Panthera onca,563517,76,Atlantic forest,Paraguay


In [3]:
data_cp = data.copy()
data_cp.isnull().sum()

Event_ID                            0
timestamp                           0
location.long                       0
location.lat                        0
individual.taxon.canonical.name     0
tag.local.identifier                0
individual.local.identifier (ID)    0
study.name                          0
country                             0
dtype: int64

## Fixing features names

In [4]:
# Fix features names Event_ID,timestamp,location.long,location.lat,individual.taxon.canonical.name,tag.local.identifier,individual.local.identifier (ID),study.name,country
data_cp = data_cp.rename(columns={'Event_ID':'event_id','timestamp':'timestamp','location.long':'location_long','location.lat':'location_lat','individual.taxon.canonical.name':'individual_taxon_canonical_name','tag.local.identifier':'tag_local_identifier','individual.local.identifier (ID)':'individual_local_identifier_ID','study.name':'study_name','country':'country'})

# Print features names
data_cp.columns


Index(['event_id', 'timestamp', 'location_long', 'location_lat',
       'individual_taxon_canonical_name', 'tag_local_identifier',
       'individual_local_identifier_ID', 'study_name', 'country'],
      dtype='object')

# Preprocess timestamp

In [5]:
# Parsing the timestamp column into a datetime format
data_cp['timestamp'] = pd.to_datetime(data_cp['timestamp'])

# Extract time of day, day of the week, month, etc., for temporal features.
data_cp['hour'] = data_cp['timestamp'].dt.hour
data_cp['day'] = data_cp['timestamp'].dt.day
data_cp['month'] = data_cp['timestamp'].dt.month
data_cp['year'] = data_cp['timestamp'].dt.year
data_cp['dayofweek'] = data_cp['timestamp'].dt.dayofweek
data_cp['date'] = data_cp['timestamp'].dt.date

# Show unique values for day, month and year
# print(data_cp['day'].unique())
# print(data_cp['month'].unique())
# print(data_cp['year'].unique())
# print(data_cp['country'].unique())

C:\Users\lmars\AppData\Local\Temp\ipykernel_11772\3021486648.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_cp['timestamp'] = pd.to_datetime(data_cp['timestamp'])


## Identify Outliers

In [6]:
# data_cp = data_cp.drop(['event_id', 'individual_local_identifier_ID', 'tag_local_identifier'], axis=1)

data_cp.columns

Index(['event_id', 'timestamp', 'location_long', 'location_lat',
       'individual_taxon_canonical_name', 'tag_local_identifier',
       'individual_local_identifier_ID', 'study_name', 'country', 'hour',
       'day', 'month', 'year', 'dayofweek', 'date'],
      dtype='object')

# Geospatial Data Processing

## Calculating distances between consecutive events.
### Calculating the distance between points using the Haversine Formula and group the data by tag_local_identifier

In [7]:
data_fe = data_cp.copy()

data_fe['timestamp'] = pd.to_datetime(data_fe['timestamp'])

# Define Haversine formula for distance calculation
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

# Function to calculate distance only for the same animal
def calculate_distances_for_animal(group):
    group = group.sort_values('timestamp')
    group['distance'] = haversine(
        group['location_lat'].shift(),
        group['location_long'].shift(),
        group['location_lat'],
        group['location_long']
    )
    return group

data_fe = data_fe.groupby('tag_local_identifier').apply(calculate_distances_for_animal)

# Convert distance to kilometers
data_fe['distance'] = data_fe['distance'] / 1000
data_fe['distance'] = data_fe['distance'].round(3)

data_fe.head()
# data_fe.to_csv('DataS1/jaguar_movement_data_cleaned.csv', index=False)

event_id           timestamp  location_long  \
tag_local_identifier                                                       
35957                131878  131879.0 2015-10-12 08:00:00     -57.502841   
                     131901  131902.0 2015-10-13 07:00:00     -57.500805   
                     131917  131918.0 2015-10-14 00:00:00     -57.498957   
                     131926  131927.0 2015-10-14 09:00:00     -57.497686   
                     131931  131932.0 2015-10-14 14:00:00     -57.497668   

                             location_lat individual_taxon_canonical_name  \
tag_local_identifier                                                        
35957                131878    -16.880885                   Panthera onca   
                     131901    -16.883557                   Panthera onca   
                     131917    -16.885097                   Panthera onca   
                     131926    -16.889175                   Panthera onca   
                     131931    -16.889110                   Panthera onca   

                            tag_local_identifier  \
tag_local_identifier                               
35957                131878                35957   
                     131901                35957   
                     131917                35957   
                     131926                35957   
                     131931                35957   

                             individual_local_identifier_ID     study_name  \
tag_local_identifier                                                         
35957                131878                             117  Jaguar_Taiama   
                     131901                             117  Jaguar_Taiama   
                     131917                             117  Jaguar_Taiama   
                     131926                             117  Jaguar_Taiama   
                     131931                             117  Jaguar_Taiama   

                            country  hour  day  month  year  dayofweek  \
tag_local_identifier                                                     
35957                131878  Brazil     8   12     10  2015          0   
                     131901  Brazil     7   13     10  2015          1   
                     131917  Brazil     0   14     10  2015          2   
                     131926  Brazil     9   14     10  2015          2   
                     131931  Brazil    14   14     10  2015          2   

                                   date  distance  
tag_local_identifier                               
35957                131878  2015-10-12       NaN  
                     131901  2015-10-13     0.368  
                     131917  2015-10-14     0.261  
                     131926  2015-10-14     0.473  
                     131931  2015-10-14     0.007

# Creating velocity, direction, and movement features based on location data (location.long, location.lat).

3. Labeling the Data
 Behavior Labeling:
 Manually label a subset of data with different behaviors (e.g., movement, hunting, resting).
 Consider cross-referencing domain knowledge or complementary data to infer labels.

In [8]:
# Creating velocity, direction, and movement features based on location data (location.long, location.lat).
# time_diff = (data_cp['timestamp'] - data_cp['timestamp'].shift()).dt.total_seconds()

# Ensuring the distance is in meters and the time differece is in seconds
time_diff = data_fe['time_diff'] = data_fe['timestamp'].diff().dt.total_seconds() / 3600

data_fe['distance'] = data_fe['distance'] * 1000
time_diff_seconds = data_fe['time_diff'] * 3600

# print(data_fe['time_diff'].head())

data_fe['velocity'] = data_fe['distance'] / time_diff_seconds
data_fe['velocity'] = data_fe['velocity'].round(4)
data_fe = data_fe.dropna(subset=[ 'distance', 'velocity'])

data_fe['direction'] = np.arctan2(data_fe['location_long'] - data_fe['location_long'].shift(), data_fe['location_lat'] - data_fe['location_lat'].shift())

# movement, hunting, resting, or slowing down
data_fe['movement'] = np.where(data_fe['velocity'] > 0.1, 'movement', 'resting')
data_fe['movement'] = np.where(data_fe['velocity'] > 0.5, 'movement', 'hunting')
data_fe['movement'] = np.where(data_fe['velocity'] < 0.1, 'movement', 'slowing down')

# data_cp = data_cp.dropna()
print(data_fe[['velocity', 'movement']])
# print(data_fe.head())

                             velocity  movement
tag_local_identifier                           
35957                131901    0.0044  movement
                     131917    0.0043  movement
                     131926    0.0146  movement
                     131931    0.0004  movement
                     131956    0.0078  movement
...                               ...       ...
TIAGO                105877    0.0144  movement
                     105967    0.0018  movement
                     105991    0.0197  movement
                     106108    0.0127  movement
                     106112    0.0027  movement

[4889 rows x 2 columns]


## Derive acceleration or changes in movement direction.

In [9]:
# Calculate acceleration as the change in velocity over time
data_fe['acceleration'] = data_fe['velocity'].diff() / data_fe['time_diff']

# Calculate change in direction
data_fe['change_in_direction'] = data_fe['direction'].diff()

# Drop rows with NaN values in acceleration and change_in_direction
data_fe = data_fe.dropna(subset=['acceleration', 'change_in_direction'])

# Round the values for better readability
data_fe['acceleration'] = data_fe['acceleration'].round(4)
data_fe['change_in_direction'] = data_fe['change_in_direction'].round(4)

# Display the first few rows of the updated dataframe
data_fe[['acceleration', 'change_in_direction']].head()

acceleration  change_in_direction
tag_local_identifier                                          
35957                131926        0.0011               0.5739
                     131931       -0.0028              -2.5793
                     131956        0.0003               0.9889
                     131963       -0.0010              -2.4232
                     131972        0.0007               0.9281

## Encoding Categorical Variables

In [10]:
# Encode tag_local_identifier to numerical values

data_fe['tag_local_identifier'] = pd.Categorical(data_fe['tag_local_identifier'])
data_fe['tag_local_identifier'] = data_fe['tag_local_identifier'].cat.codes

print(data_fe['tag_local_identifier'].unique())
# data_encoded = pd.get_dummies(data_fe, columns=['tag_local_identifier'])
# data_encoded.columns
# data_encoded['tag_local_identifier'].unique()

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104]


## Now we want to get temporal features from Paraguay, Brazil, Costa Rica, Argentina and Mexico between 1/1/1999 and 31/12/2016

In [11]:
# Get World  Climate data
import kagglehub

path = kagglehub.dataset_download("christopherlemke/monthly-climat-reports-from-stations-worldwide")

reports_data = pd.read_csv(path + '/dwd-cdc_CLIMAT_reports_stations_ww.csv')
stations_data = pd.read_csv(path + '/dwd-cdc_station_data_ww.csv')

# Save the data to a csv file
reports_data.to_csv('DataS1/monthly-climat-reports-from-stations-worldwide.csv', index=False)
stations_data.to_csv('DataS1/dwd-cdc_station_data_ww.csv', index=False)

print(reports_data.shape)
print(stations_data.shape)

reports_data = reports_data.sample(SAMPLE_SIZE)
# stations_data = stations_data.sample(SAMPLE_SIZE)

# print(reports_data.head())
# print(stations_data.head())

# print("Path to dataset files:", path)

c:\Users\lmars\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(553652, 127)
(4169, 6)


### Preprocessing Reports data

In [12]:
def farenheit_to_celsius(farenheit):
    return (farenheit - 32) * 5.0/9.0

2. Feature Engineering
 Environmental Features:
 Integrate meteorological data (e.g., temperature, rainfall) if available.
 Integrate terrain data (e.g., elevation, land cover type) if available.
 Temporal Features:
 Extract time of day, day of week, and season from the timestamp.
 Identify specific behavioral periods like night vs. day.<br><br>

In [13]:
# Remove columns
reports_data_dropped = reports_data.drop(['Po', 'Po.1', 'P', 'P.1', 'T.1', 'st', 'st.1', 
                                          'Tx.1', 'Tn.1', 
                                          'e', 'e.1', 
                                          'R1.1', 'nr.1', 'S1.1', 'Rd', 'mp', 'mT', 'mTx', 'mTn', 'me', 'mR', 'mS', 
                                            #   'sn', 'sn.1', 'sn.2', 'sn.3', 'sn.4', 'sn.5', 'sn.6', 'sn.7', 'sn.8', 'sn.9',
                                            # 'G1', 'G1.1', 'G1.2', 'G1.3', 'G1.4', 'G1.5', 'G1.6', 'G1.7', 'G1.8',
                                            # 'G2', 'G2.1', 'G2.2', 'G2.3', 'G2.4', 'G2.5', 'G2.6', 'G2.7', 'G2.8', 'G2.9',
                                            # 'G3', 'G3.1', 'G3.2', 'G3.3', 'G3.4', 'G3.5', 'G3.6', 'G3.7', 'G3.8', 'G3.9',
                                            # 'G4', 'G4.1', 'G4.2', 'G4.3', 'G4.4', 'G4.5', 'G4.6', 'G4.7',
                                            'Yb', 'Yc', 'P', 'YP', 'YR', 'YS', 'YT', 'YTx',
                                            'Ye', 'G4', 'Txd', 'yx', 'Tnd', 'Tax', 'Tan' ], axis=1)

# Drop columns with no data
reports_data_dropped = reports_data_dropped.dropna(axis=1, how='all')

# Rename columns
reports_data_renamed = reports_data_dropped.rename(columns={'IIiii':'station_id', 'T':'monthly_mean_air_temperature', 
                                            'Tx':'mean_daily_maximum_air_temperature', 
                                            'Tn':'mean_daily_minimum_air_temperature', 
                                            'R1':'total_precipitation_month', 'S1':'total_sunshine_month', 
                                            'ps':'percentage_total_sunshine_duration_relative_normal', 
                                            'P0':'monthly_mean_pressure_station_level', 'e':'mean_vapor_pressure_month', 
                                            'nr':'number_days_month_precipitation', 
                                            'yP':'missing_years_air_pressure', 'yR':'missing_years_precipitation', 'yS':'missing_years_sunshine_duration', 
                                            'yT':'missing_years_mean_air_temperature', 'yTx':'missing_years_mean_extreme_air_temperature', 'ye':'missing_years_vapor_pressure', 
                                            'T25':'days_month_maximum_air_temperature_25', 'T30':'days_month_maximum_air_temperature_30', 
                                            'T35':'days_month_maximum_air_temperature_35', 'T40':'days_month_maximum_air_temperature_40', 
                                            'Tn0':'days_month_minimum_air_temperature_0', 'Tx0':'days_month_maximum_air_temperature_0', 'R01':'days_month_precipitation_1', 
                                            'R05':'days_month_precipitation_5', 'R10':'days_month_precipitation_10', 'R50':'days_month_precipitation_50', 
                                            'R100':'days_month_precipitation_100', 'R150':'days_month_precipitation_150', 's00':'days_month_snow_depth_0', 
                                            's01':'days_month_snow_depth_1', 's10':'days_month_snow_depth_10', 's50':'days_month_snow_depth_50', 'f10':'days_month_wind_speed_10', 
                                            'f20':'days_month_wind_speed_20', 'f30':'days_month_wind_speed_30', 'V1':'days_month_visibility_50', 'V2':'days_month_visibility_100', 
                                            'V3':'days_month_visibility_1000', 'yn': 'day_lowest_daily_mean_air_temperature_month', 'yax': 'day_highest_daily_mean_air_temperature_month', 'yan':'day_lowest_air_tempreature_month',
                                            'Rx':'highest_daily_amount_precipitation_month_tenths_mm',  
                                            # 'yr':'day_highest_daily_amount_precipitation_month'
                                            })

# Remove rows with missing values in [year]
reports_data_renamed = reports_data_renamed.dropna(subset=['year'])

# Enconde Year, Month, and Station ID to integer
reports_data_renamed['year'] = reports_data_renamed['year'].apply(lambda x: int(x) if pd.notnull(x) else x)
reports_data_renamed['month'] = reports_data_renamed['month'].apply(lambda x: int(x) if pd.notnull(x) else x)
reports_data_renamed['station_id'] = reports_data_renamed['station_id'].apply(lambda x: int(x) if pd.notnull(x) else x)
reports_data_renamed['sn'] = reports_data_renamed['sn'].apply(lambda x: int(x) if pd.notnull(x) else x)

# Convert temperature features to Celsius
# °C = (°F - 32) × 5/9
reports_data_renamed['monthly_mean_air_temperature'] = farenheit_to_celsius(reports_data_renamed['monthly_mean_air_temperature']).round(2)
reports_data_renamed['mean_daily_maximum_air_temperature'] =  farenheit_to_celsius(reports_data_renamed['mean_daily_maximum_air_temperature']).round(2)
reports_data_renamed['mean_daily_minimum_air_temperature'] = farenheit_to_celsius(reports_data_renamed['mean_daily_minimum_air_temperature']).round(2)

# Check for missing values in temperature features
# print(reports_data_renamed[['monthly_mean_air_temperature', 'mean_daily_maximum_air_temperature', 'mean_daily_minimum_air_temperature']].isnull().sum())

# Fill missing values with the mean
reports_data_renamed['monthly_mean_air_temperature'] = reports_data_renamed['monthly_mean_air_temperature'].fillna(reports_data_renamed['monthly_mean_air_temperature'].mean())
reports_data_renamed['mean_daily_maximum_air_temperature'] = reports_data_renamed['mean_daily_maximum_air_temperature'].fillna(reports_data_renamed['mean_daily_maximum_air_temperature'].mean())
reports_data_renamed['mean_daily_minimum_air_temperature'] = reports_data_renamed['mean_daily_minimum_air_temperature'].fillna(reports_data_renamed['mean_daily_minimum_air_temperature'].mean())

print(reports_data_renamed[['monthly_mean_air_temperature', 'mean_daily_maximum_air_temperature', 'mean_daily_minimum_air_temperature']].isnull().sum())

# # Show null values
# print(reports_data_renamed.isnull().sum())
reports_data_renamed.to_csv('DataS1/monthly-climat-reports-from-stations-worldwide-cleaned.csv', index=False)

reports_data_renamed.head()


monthly_mean_air_temperature          0
mean_daily_maximum_air_temperature    0
mean_daily_minimum_air_temperature    0
dtype: int64


,year,month,station_id,G1,G1.1,G1.2,sn,monthly_mean_air_temperature,G1.3,sn.1,...,iw,fx,yfx,G4.6,Dts,Dgr,G4.7,iy,Gx,Gn
27189,2008,12,28434,1.0,2.0,3.0,1.0,39.44,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131197,2017,11,30054,1.0,2.0,3.0,1.0,85.00,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172314,2010,11,43041,1.0,2.0,3.0,0.0,116.67,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294283,2016,12,38611,1.0,2.0,3.0,0.0,-2.22,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388019,2012,8,38656,1.0,2.0,3.0,0.0,158.33,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# stations_data.dtypes

# Rename columns
# 0: Station ID, 1: Station Name, 2: Latitude, 3: Longitude, 4:Height, 5: Country
stations_data_renamed = stations_data.rename(columns={'0':'station_id', '1':'station_name', '2':'latitude', '3':'longitude', '4':'height', '5':'country'})
stations_data_renamed['station_id'] = stations_data_renamed['station_id'].str.strip()
stations_data_renamed = stations_data_renamed[:-1]

# print(stations_data_renamed['station_id'].unique())


stations_data_renamed['station_id'] = (
    stations_data_renamed['station_id']
    .str.replace(r'\D', '', regex=True)  # Remove all non-digit characters
    .pipe(pd.to_numeric, errors='coerce')  # Convert to numeric, invalid become NaN
    .astype('Int64')  # Convert to pandas' nullable Int64 type
)


# stations_data_renamed['station_id'] = stations_data_renamed['station_id'].apply(lambda x: int(x) if pd.notnull(x) else x)
# stations_data_renamed.head()
# stations_data_renamed.info()

In [15]:
# Merge the reports and stations data by station_id
import sqlite3


data_merged = pd.merge(reports_data_renamed, stations_data_renamed, on='station_id')

conn = sqlite3.connect('jaguar_data.db')

# data_fe['country'].unique()
data_merged['country'] = data_merged['country'].str.strip()
data_merged['country'].unique()
# data_merged['country'].isnull().sum()

print(data_merged.shape)
print(data_fe.shape)
# data_merged.head()

# Save data to SQLite database
reports_data_renamed.to_sql('reports', conn, if_exists='replace', index=False)
stations_data_renamed.to_sql('stations', conn, if_exists='replace', index=False)
data_fe.to_sql('data_fe', conn, if_exists='replace', index=False)

print("Data successfully loaded into SQLite database!")

# Merge data_merged with data_fe only by the countries in data_fe
data_merged_countries = pd.merge(data_fe, data_merged, on='country')

# data_merged_countries = pd.merge(data_merged, data_fe, on='country', how='inner')
# data_merged_countries = data_merged[data_merged['country'].isin(data_fe['country'].unique())]
# data_merged_countries.head()

conn.close()
print("Database connection closed.")

# Save
data_merged_countries.to_csv('DataS1/jaguar_movement_with_countries_climate_data.csv', index=False)

(4673, 96)
(4887, 22)
Data successfully loaded into SQLite database!
Database connection closed.


In [16]:
avg_prec = pd.read_csv('DataS1/average-precipitation-per-year.csv')
avg_prec.drop(['Code'], axis=1, inplace=True)
avg_prec.rename(columns={'Entity':'country', 'Year':'year'}, inplace=True)
avg_prec.head()

# Merge the two datasets by Country
data_merged = pd.merge(data_fe, avg_prec, on=['country', 'year'], how='left')
data_merged.head()

# Save
data_merged.to_csv('DataS1/jaguar_movement_data_cleaned.csv', index=False)

4. Model Selection
 Choose Algorithms:
 Evaluate models like Random Forest, Gradient Boosting Machines (e.g., XGBoost), or neural networks (RNN/LSTM for sequential data).
 Decide on the best model based on data type and problem complexity.
 Split Data:
 Split the data into training and testing sets (e.g., 80/20 or 70/30 split).

5. Model Training and Evaluation
 Train Models:
 Train selected models using the preprocessed data.
 Use cross-validation to tune model parameters.
 Evaluate Model:
 Evaluate performance using metrics like accuracy, precision, recall, and F1-score.
 If the dataset is imbalanced, apply techniques like SMOTE to balance it.

6. Cross-Referencing with Complementary Data
 Integrate Meteorological Data:
 Merge weather data (e.g., temperature, humidity) with animal tracking data.
 Integrate Terrain Data:
 Merge terrain data (e.g., elevation, land use) with animal tracking data.
 Behavioral Biology Data:
 Incorporate knowledge from related behavioral biology studies (if available).

7. Model Deployment
 Model Evaluation: Confirm that the model generalizes well to new or unseen data.
 Deploy Model:
 Prepare the model for deployment in a real-time or batch setting for wildlife tracking.
 Implement a user interface or tool to apply the model to new data.